In this notebook we perform ensembling on the prediction obtained from BERT, ALBERT, RoBERTa, DistilBERT. 

After training, every model generates a file called. 'predictions'. Contents of this file include a predicted answer for every question. 
We predictions from all of the above models to create an ensemble based on majority voting.  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json 
  
f = open('/content/drive/MyDrive/all_predictions/predictions_albert.json',) 
data_albert = json.load(f) 
f.close() 

g = open('/content/drive/MyDrive/all_predictions/predictions_bert.json',) 
data_bert = json.load(g) 
g.close() 

h = open('/content/drive/MyDrive/all_predictions/predictions_distilbert.json',) 
data_distilbert = json.load(h) 
h.close() 

i = open('/content/drive/MyDrive/all_predictions/predictions_roberta.json',) 
data_roberta = json.load(i) 
i.close() 

In [ ]:
ids = []
for key in data_albert:
    ids.append(key)

In [ ]:
answers_albert = []
answers_bert = []
answers_distilbert = []
answers_roberta = []

for id in ids:
    answers_albert.append(data_albert[id])

for id in ids:
    answers_bert.append(data_bert[id])

for id in ids:
    answers_distilbert.append(data_distilbert[id])

for id in ids:
    answers_roberta.append(data_roberta[id])  

In [ ]:
all_preds = [] 
for i in range(len(ids)):
    all_preds.append([ids[i], answers_albert[i], answers_bert[i], answers_distilbert[i], answers_roberta[i]])

In [ ]:
all_preds[0]

['56ddde6b9a695914005b9628', 'France', 'France', 'France', 'France.']

In [ ]:
import pandas as pd
df = pd.DataFrame(all_preds, columns = ['ids', 'albert answers', 'bert answers', 'distilbert answers', 'roberta answers'])
df.head()

,ids,albert answers,bert answers,distilbert answers,roberta answers
0,56ddde6b9a695914005b9628,France,France,France,France.
1,56ddde6b9a695914005b9629,10th and 11th centuries,10th and 11th centuries,10th and 11th centuries,10th and 11th centuries
2,56ddde6b9a695914005b962a,"Denmark, Iceland and Norway","Denmark, Iceland and Norway","Denmark, Iceland and Norway","Denmark, Iceland and Norway"
3,56ddde6b9a695914005b962b,,Rollo,Rollo,
4,56ddde6b9a695914005b962c,10th,10th,10th,10th


In [ ]:
df.shape

(11873, 5)

In [ ]:
from collections import Counter
import operator

fin_preds = []
for i in range(len(ids)):
    d = Counter(all_preds[i][1:])
    fin_preds.append([ids[i], max(d.items(), key=operator.itemgetter(1))[0]])

In [ ]:
df_fin = pd.DataFrame(fin_preds, columns = ['ids', 'ensembled answers'])
df_fin.head()

,ids,ensembled answers
0,56ddde6b9a695914005b9628,France
1,56ddde6b9a695914005b9629,10th and 11th centuries
2,56ddde6b9a695914005b962a,"Denmark, Iceland and Norway"
3,56ddde6b9a695914005b962b,
4,56ddde6b9a695914005b962c,10th


Replace no answers with roberta answers


In [ ]:
fin_preds_dict = {}

for l in fin_preds:
    fin_preds_dict[l[0]] = l[1]
    

In [ ]:
for i in data_roberta:
    if data_roberta[i]=='':
        fin_preds_dict[i] = data_roberta[i]

In [ ]:
processed_fin_preds = []

for key in fin_preds_dict.keys():
    processed_fin_preds.append([key, fin_preds_dict[key]])

In [ ]:
processed_fin_preds[0]

['56ddde6b9a695914005b9628', 'France']

In [ ]:
!pip install transformers
!pip install tensorboardX

     |████████████████████████████████| 1.4MB 9.3MB/s 
     |████████████████████████████████| 2.9MB 30.6MB/s 
     |████████████████████████████████| 890kB 49.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=948a7a6d01d65c4aaab8ba8e25d5d6b24c22bd47ca923360889f48ac599b1b71
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 317kB 9.3MB/s 


In [ ]:
import pickle
with open('/content/drive/MyDrive/all_predictions/examples.pickle', 'rb') as handle:
    examples = pickle.load(handle)

with open('/content/drive/MyDrive/all_predictions/predictions.pickle', 'rb') as handle:
    predictions = pickle.load(handle)

In [ ]:
import collections
import json
import math
import re
import string

from transformers import BasicTokenizer

# from ...utils import logging

In [ ]:
def compute_f1(a_gold, a_pred):
    gold_toks = get_tokens(a_gold)
    pred_toks = get_tokens(a_pred)
    common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
    num_same = sum(common.values())
    if len(gold_toks) == 0 or len(pred_toks) == 0:
        # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
        return int(gold_toks == pred_toks)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(pred_toks)
    recall = 1.0 * num_same / len(gold_toks)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

In [ ]:
def get_raw_scores(examples, preds):
    """
    Computes the exact and f1 scores from the examples and the model predictions
    """
    exact_scores = {}
    f1_scores = {}

    for example in examples:
        qas_id = example.qas_id
        gold_answers = [answer["text"] for answer in example.answers if normalize_answer(answer["text"])]

        if not gold_answers:
            # For unanswerable questions, only correct answer is empty string
            gold_answers = [""]

        if qas_id not in preds:
            print("Missing prediction for %s" % qas_id)
            continue

        prediction = preds[qas_id]
        exact_scores[qas_id] = max(compute_exact(a, prediction) for a in gold_answers)
        f1_scores[qas_id] = max(compute_f1(a, prediction) for a in gold_answers)

    return exact_scores, f1_scores


In [ ]:
def apply_no_ans_threshold(scores, na_probs, qid_to_has_ans, na_prob_thresh):
    new_scores = {}
    for qid, s in scores.items():
        pred_na = na_probs[qid] > na_prob_thresh
        if pred_na:
            new_scores[qid] = float(not qid_to_has_ans[qid])
        else:
            new_scores[qid] = s
    return new_scores

In [ ]:
def make_eval_dict(exact_scores, f1_scores, qid_list=None):
    if not qid_list:
        total = len(exact_scores)
        return collections.OrderedDict(
            [
                ("exact", 100.0 * sum(exact_scores.values()) / total),
                ("f1", 100.0 * sum(f1_scores.values()) / total),
                ("total", total),
            ]
        )
    else:
        total = len(qid_list)
        return collections.OrderedDict(
            [
                ("exact", 100.0 * sum(exact_scores[k] for k in qid_list) / total),
                ("f1", 100.0 * sum(f1_scores[k] for k in qid_list) / total),
                ("total", total),
            ]
        )

In [ ]:
def merge_eval(main_eval, new_eval, prefix):
    for k in new_eval:
        main_eval["%s_%s" % (prefix, k)] = new_eval[k]

In [ ]:
def find_best_thresh_v2(preds, scores, na_probs, qid_to_has_ans):
    num_no_ans = sum(1 for k in qid_to_has_ans if not qid_to_has_ans[k])
    cur_score = num_no_ans
    best_score = cur_score
    best_thresh = 0.0
    qid_list = sorted(na_probs, key=lambda k: na_probs[k])
    for i, qid in enumerate(qid_list):
        if qid not in scores:
            continue
        if qid_to_has_ans[qid]:
            diff = scores[qid]
        else:
            if preds[qid]:
                diff = -1
            else:
                diff = 0
        cur_score += diff
        if cur_score > best_score:
            best_score = cur_score
            best_thresh = na_probs[qid]

    has_ans_score, has_ans_cnt = 0, 0
    for qid in qid_list:
        if not qid_to_has_ans[qid]:
            continue
        has_ans_cnt += 1

        if qid not in scores:
            continue
        has_ans_score += scores[qid]

    return 100.0 * best_score / len(scores), best_thresh, 1.0 * has_ans_score / has_ans_cnt


In [ ]:
def find_all_best_thresh_v2(main_eval, preds, exact_raw, f1_raw, na_probs, qid_to_has_ans):
    best_exact, exact_thresh, has_ans_exact = find_best_thresh_v2(preds, exact_raw, na_probs, qid_to_has_ans)
    best_f1, f1_thresh, has_ans_f1 = find_best_thresh_v2(preds, f1_raw, na_probs, qid_to_has_ans)
    main_eval["best_exact"] = best_exact
    main_eval["best_exact_thresh"] = exact_thresh
    main_eval["best_f1"] = best_f1
    main_eval["best_f1_thresh"] = f1_thresh
    main_eval["has_ans_exact"] = has_ans_exact
    main_eval["has_ans_f1"] = has_ans_f1

In [ ]:
def find_best_thresh(preds, scores, na_probs, qid_to_has_ans):
    num_no_ans = sum(1 for k in qid_to_has_ans if not qid_to_has_ans[k])
    cur_score = num_no_ans
    best_score = cur_score
    best_thresh = 0.0
    qid_list = sorted(na_probs, key=lambda k: na_probs[k])
    for _, qid in enumerate(qid_list):
        if qid not in scores:
            continue
        if qid_to_has_ans[qid]:
            diff = scores[qid]
        else:
            if preds[qid]:
                diff = -1
            else:
                diff = 0
        cur_score += diff
        if cur_score > best_score:
            best_score = cur_score
            best_thresh = na_probs[qid]
    return 100.0 * best_score / len(scores), best_thresh

In [ ]:
def find_all_best_thresh(main_eval, preds, exact_raw, f1_raw, na_probs, qid_to_has_ans):
    best_exact, exact_thresh = find_best_thresh(preds, exact_raw, na_probs, qid_to_has_ans)
    best_f1, f1_thresh = find_best_thresh(preds, f1_raw, na_probs, qid_to_has_ans)

    main_eval["best_exact"] = best_exact
    main_eval["best_exact_thresh"] = exact_thresh
    main_eval["best_f1"] = best_f1
    main_eval["best_f1_thresh"] = f1_thresh

In [ ]:
def squad_evaluate(examples, preds, no_answer_probs=None, no_answer_probability_threshold=1.0):
    qas_id_to_has_answer = {example.qas_id: bool(example.answers) for example in examples}
    has_answer_qids = [qas_id for qas_id, has_answer in qas_id_to_has_answer.items() if has_answer]
    no_answer_qids = [qas_id for qas_id, has_answer in qas_id_to_has_answer.items() if not has_answer]

    if no_answer_probs is None:
        no_answer_probs = {k: 0.0 for k in preds}

    exact, f1 = get_raw_scores(examples, preds)

    exact_threshold = apply_no_ans_threshold(
        exact, no_answer_probs, qas_id_to_has_answer, no_answer_probability_threshold
    )
    f1_threshold = apply_no_ans_threshold(f1, no_answer_probs, qas_id_to_has_answer, no_answer_probability_threshold)

    evaluation = make_eval_dict(exact_threshold, f1_threshold)

    if has_answer_qids:
        has_ans_eval = make_eval_dict(exact_threshold, f1_threshold, qid_list=has_answer_qids)
        merge_eval(evaluation, has_ans_eval, "HasAns")

    if no_answer_qids:
        no_ans_eval = make_eval_dict(exact_threshold, f1_threshold, qid_list=no_answer_qids)
        merge_eval(evaluation, no_ans_eval, "NoAns")

    if no_answer_probs:
        find_all_best_thresh(evaluation, preds, exact, f1, no_answer_probs, qas_id_to_has_answer)

    return evaluation

In [ ]:
def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""

    def remove_articles(text):
        regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
        return re.sub(regex, " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def get_tokens(s):
    if not s:
        return []
    return normalize_answer(s).split()


def compute_exact(a_gold, a_pred):
    return int(normalize_answer(a_gold) == normalize_answer(a_pred))

In [ ]:
tuples_fin_preds = []
for l in processed_fin_preds:
    id, ans = l
    tuples_fin_preds.append((id, ans))


In [ ]:
from collections import OrderedDict
dict_l = OrderedDict(tuples_fin_preds)
print(dict_l)

In [ ]:
fin_res = squad_evaluate(examples, dict_l)

In [ ]:
fin_res

OrderedDict([('exact', 80.71254105954687),
             ('f1', 83.4004840855398),
             ('total', 11873),
             ('HasAns_exact', 73.4480431848853),
             ('HasAns_f1', 78.83163757550851),
             ('HasAns_total', 5928),
             ('NoAns_exact', 87.95626576955425),
             ('NoAns_f1', 87.95626576955425),
             ('NoAns_total', 5945),
             ('best_exact', 80.71254105954687),
             ('best_exact_thresh', 0.0),
             ('best_f1', 83.40048408553969),
             ('best_f1_thresh', 0.0)])